# MVP — Online Retail II (versão funcional)

Notebook pronto para executar no Google Colab. Organização similar ao checklist solicitado: carga, limpeza, EDA, engenharia de atributos (RFM), segmentação (KMeans), tarefa supervisonada (recompra 90 dias), baselines, otimização e avaliação.

Se a importação automática falhar, siga as instruções da célula de carregamento para fazer upload manual do arquivo Excel/CSV do dataset.


In [ ]:
# Instalações (Colab): descomente se necessário
!pip install --quiet ucimlrepo xgboost

import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print('Ambiente pronto. Random seed fixada:', RANDOM_STATE)


In [ ]:
# Carregamento do dataset: tentar via ucimlrepo; se falhar, peça upload manual
df = None
try:
    from ucimlrepo import fetch_ucirepo
    print('Tentando baixar via ucimlrepo (UCI)...')
    ds = fetch_ucirepo(id=502)  # id conhecido do Online Retail II
    # tentar extrair DataFrame
    if hasattr(ds.data, 'features'):
        df = ds.data.features.copy()
    else:
        try:
            df = pd.DataFrame(ds.data)
        except Exception:
            df = None
except Exception as e:
    print('ucimlrepo não funcionou automaticamente:', str(e))

if df is None:
    print('\nNão foi possível carregar automaticamente o dataset. Por favor faça upload manual do arquivo:')
    print('1) Baixe o arquivo Online Retail II (ex: Excel) do repositório UCI no seu navegador.')
    print('2) No Colab: clique em Arquivo → Upload e selecione o arquivo, ou use o código abaixo para fazer upload interativo.')
    from google.colab import files
    uploaded = files.upload()
    # tenta identificar o arquivo carregado
    for name in uploaded:
        try:
            if name.lower().endswith('.xlsx'):
                df = pd.read_excel(name)
            else:
                df = pd.read_csv(name)
            print('Arquivo carregado e lido:', name)
            break
        except Exception as e:
            print('Falha ao ler', name, str(e))

if df is None:
    raise RuntimeError('Nenhum dataset carregado. Interrompendo. Por favor carregue o arquivo Online Retail II no Colab e execute novamente.')

print('Dimensão inicial do dataset:', df.shape)
df.head()


In [ ]:
# Limpeza e preparação inicial
df.columns = [c.strip() for c in df.columns]
if 'InvoiceDate' in df.columns:
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
if 'Quantity' in df.columns and 'UnitPrice' in df.columns:
    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

print('Valores nulos por coluna:')
print(df.isnull().sum())

if 'CustomerID' in df.columns:
    df = df.dropna(subset=['CustomerID'])
    df['CustomerID'] = df['CustomerID'].astype(int)

if 'InvoiceNo' in df.columns:
    df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
if 'Quantity' in df.columns:
    df = df[df['Quantity']>0]

print('Dimensão após limpeza:', df.shape)
df.head()


In [ ]:
# Engenharia RFM
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
}).reset_index()
rfm.columns = ['CustomerID','Recency','Frequency','Monetary']
rfm.describe()


In [ ]:
# Segmentação com KMeans (usar log1p + StandardScaler)
from sklearn.preprocessing import StandardScaler
X_rfm = rfm[['Recency','Frequency','Monetary']].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(np.log1p(X_rfm))

scores = {}
for k in range(2,7):
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE)
    labels = km.fit_predict(X_scaled)
    scores[k] = silhouette_score(X_scaled, labels)

print('Silhouette scores por k:', scores)
best_k = max(scores, key=scores.get)
print('Melhor k:', best_k)
kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_STATE)
rfm['Cluster'] = kmeans.fit_predict(X_scaled)
rfm.groupby('Cluster').agg({'Recency':'median','Frequency':'median','Monetary':'median','CustomerID':'count'})


In [ ]:
# Definição do target supervisonado: recompra em 90 dias
max_date = df['InvoiceDate'].max()
cutoff_date = max_date - pd.Timedelta(days=180)
label_window = pd.Timedelta(days=90)

df_features = df[df['InvoiceDate'] <= cutoff_date].copy()
df_labels = df[(df['InvoiceDate'] > cutoff_date) & (df['InvoiceDate'] <= cutoff_date + label_window)].copy()
feature_customers = set(df_features['CustomerID'].unique())
label_customers = set(df_labels['CustomerID'].unique())

snapshot_ref = cutoff_date + pd.Timedelta(days=1)
rfm_feat = df_features.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_ref - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
}).reset_index()
rfm_feat.columns = ['CustomerID','Recency','Frequency','Monetary']
rfm_feat['target_repurchase_90d'] = rfm_feat['CustomerID'].apply(lambda x: 1 if x in label_customers else 0)

print('Tamanho features:', rfm_feat.shape)
print(rfm_feat['target_repurchase_90d'].value_counts())


In [ ]:
# Split treino/teste e escala
X = rfm_feat[['Recency','Frequency','Monetary']]
y = rfm_feat['target_repurchase_90d']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
scaler2 = StandardScaler()
X_train_scaled = scaler2.fit_transform(np.log1p(X_train))
X_test_scaled = scaler2.transform(np.log1p(X_test))
print('Train:', X_train.shape, 'Test:', X_test.shape)


In [ ]:
# Baselines: Logistic Regression e RandomForest
lr = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
t0 = time.time(); lr.fit(X_train_scaled, y_train); t1 = time.time()
y_pred_lr = lr.predict(X_test_scaled)
y_prob_lr = lr.predict_proba(X_test_scaled)[:,1]
print('LR time (s):', t1-t0)
print('LR - Accuracy:', accuracy_score(y_test,y_pred_lr), 'ROC AUC:', roc_auc_score(y_test,y_prob_lr))

rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
t0 = time.time(); rf.fit(X_train, y_train); t1 = time.time()
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:,1]
print('RF time (s):', t1-t0)
print('RF - Accuracy:', accuracy_score(y_test,y_pred_rf), 'ROC AUC:', roc_auc_score(y_test,y_prob_rf))


In [ ]:
# Exemplos de RandomizedSearch (curto) para RandomForest
param_dist = {
    'n_estimators': [50,100,200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2,5,10]
}
rs = RandomizedSearchCV(RandomForestClassifier(random_state=RANDOM_STATE), param_distributions=param_dist, n_iter=8, cv=3, scoring='roc_auc', random_state=RANDOM_STATE, n_jobs=-1)
t0 = time.time(); rs.fit(X_train, y_train); t1 = time.time()
print('RandomizedSearch time (s):', t1-t0)
print('Best params:', rs.best_params_)
best_rf = rs.best_estimator_
y_pred_best = best_rf.predict(X_test)
y_prob_best = best_rf.predict_proba(X_test)[:,1]
print('Best RF - Accuracy:', accuracy_score(y_test,y_pred_best), 'ROC AUC:', roc_auc_score(y_test,y_prob_best))


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
print(classification_report(y_test, y_pred_best))
cm = confusion_matrix(y_test, y_pred_best)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion Matrix - Best RF')
plt.show()


## Observações finais
- Fixe random_state para reprodutibilidade.
- Se quiser que eu adapte o notebook para outro dataset ou gere um notebook com **deploy** (por exemplo, exportar modelo para o GCP/AWS ou salvar artefatos no Drive), eu faço.
